In [43]:
records = SeqIO.parse('../data/sequences.fasta.aln', 'fasta')
np.unique([len(list(r.seq)) for r in records])

array([414])

In [44]:
alns = getRecordAlignments('../data/sequences.fasta.aln')
azo = getRecordAlignments('../data/sequencesLongLabels.fasta')

In [56]:
azo['001'][108:118]

['F', 'L', 'E', 'E', 'E', 'G', 'A', 'Y', 'S', 'D']

In [57]:
print(alns['001'][193:203])

['F', 'L', 'E', 'E', 'E', 'G', 'A', 'Y', 'S', 'D']


In [2]:
from collections import defaultdict
import pandas as pd
from Bio import SearchIO, SeqIO

hmmout = '../sout.txt'
hitsout = 'hits.fasta'
pepite_db = '/home/robaina/cleangenomes/results/Marref_V6.faa'

attribs = ['id', 'bias', 'bitscore', 'description']
hits = defaultdict(list)

with open(hmmout) as handle:
    for queryresult in SearchIO.parse(handle, 'hmmer3-tab'):
      for hit in queryresult.hits:
        for attrib in attribs:
          hits[attrib].append(getattr(hit, attrib))

hits_df = pd.DataFrame.from_dict(hits)

# Write hit fasta
hit_records = [record for record in SeqIO.parse(pepite_db, 'fasta')
               if record.id in hits_df.id.values]

with open(hitsout, 'w') as out_handle:
     SeqIO.write(hit_records, out_handle, 'fasta')

In [30]:
from Bio import SeqIO

input_fasta = '/home/robaina/cleangenomes/results/Marref_V6.fasta'

hit_records = [record for record in SeqIO.parse(input_fasta, 'fasta')
               if record.id in hits_df.id.values]

with open('hits.fasta', 'w') as out_handle:
    # for record in SeqIO.parse(input_fasta, 'fasta'):
    #     if record.id in hits_df.id.values:
     SeqIO.write(hit_records, out_handle, 'fasta')

# SeqIO.write(selected_seqs, out_handle, "fasta")

In [5]:
import os

os.path.splitext('/home/robaina/cleangenomes/results/Marref_V6.fasta')

('/home/robaina/cleangenomes/results/Marref_V6', '.fasta')